# Some startup code

import packages needed & load dataset

In [1]:
from sklearn import svm, datasets, preprocessing
import numpy as np
from loaddata import get_instances_from_csv
from startEvaluation import evaluation

import sys
import threading
import time
from contextlib import contextmanager

DATAPATH = "../sub_datasets/subset_0.csv"
DATAPATH1 = "../sub_datasets/subset_1.csv"
RANDOM_SEED=42
TRAINROWS = 50000
EVALROWS = 50000

training_data = get_instances_from_csv(DATAPATH, "train", numrows=TRAINROWS)
eval_data = get_instances_from_csv(DATAPATH1,"train", numrows=EVALROWS)
print("training data  : %s rows"%training_data[0].shape[0])
print("evaluation data: %s rows"%eval_data[0].shape[0])

kernels = ['rbf', 'linear', 'sigmoid', 'poly']

scaler = preprocessing.StandardScaler()
scaler.fit(training_data[0])
print(scaler)

def print_prediction(training_data, eval_data, *args, **kwargs):

    print("start training")
    svc = svm.SVC(random_state=RANDOM_SEED, cache_size=2000, max_iter=2000, *args, **kwargs).fit(training_data[0], training_data[1])
    print("finished training svm %s"%svc)
    evt = evaluation(training_data[1], svc.predict(training_data[0]))
    print("accuracy on traning data:")
    evt.print_only_accuracy()
    eve = evaluation(eval_data[1], svc.predict(eval_data[0]))
    print("accuracy on evaluation data:")
    eve.print_only_accuracy()


/usr/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (0,1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


training data  : 49999 rows
evaluation data: 49999 rows
StandardScaler(copy=True, with_mean=True, with_std=True)


## Basic SVM
As you can see the print_prediction function fits the SVM-Model to the training data and prints a very simple evaluation.

In [ ]:
print_prediction(training_data, eval_data)

start training


/usr/lib/python3.5/site-packages/sklearn/svm/base.py:224: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


finished training svm SVC(C=1.0, cache_size=2000, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=2000, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)
accuracy on traning data:
0.830856617132
accuracy on evaluation data:
0.830146602932



## Lets use different kernels and see how the perform
Currently there are 3 Kernels implemented by sklearn that we can test. (Of course we could also implement own kernels later on)

In [ ]:
for kernel in kernels:
    print(kernel)
    print_prediction(training_data, eval_data, kernel=kernel)

rbf
start training


/usr/lib/python3.5/site-packages/sklearn/svm/base.py:224: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


finished training svm SVC(C=1.0, cache_size=2000, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=2000, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)
accuracy on traning data:
0.830383274332
accuracy on evaluation data:
0.830146602932
linear
start training


/usr/lib/python3.5/site-packages/sklearn/svm/base.py:224: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


finished training svm SVC(C=1.0, cache_size=2000, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=2000, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)
accuracy on traning data:
0.765271305426
accuracy on evaluation data:
0.722544450889
sigmoid
start training


/usr/lib/python3.5/site-packages/sklearn/svm/base.py:224: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


finished training svm SVC(C=1.0, cache_size=2000, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='sigmoid',
  max_iter=2000, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)
accuracy on traning data:
0.638481341055
accuracy on evaluation data:
0.57537400748
poly
start training


The RBF kernel seems to perform slightly better then the linear kernel. Both are far better then the sigmoid-kernel.

## Lets also check, if scaling the data improves the classification performance
Luckily sklearn also provides features for scaling data, the standart scaling function from sklearn scales the data in a way that variance == 1 and mean == 0.

In [ ]:
training_data_scaled = [scaler.transform(training_data[0]), training_data[1]]
eval_data_scaled = [scaler.transform(eval_data[0]), eval_data[1]]
for kernel in kernels:
    print(kernel)
    print_prediction(training_data_scaled, eval_data_scaled, kernel=kernel)

## Parameter Variations

In [ ]:
for kernel in kernels:
    for C in np.logspace(-2, 0, num=5):
        print_prediction(training_data, eval_data, kernel=kernel, C=C)

for kernel in kernels:
    for gamma in np.logspace(-2, 0, num=5):
        print_prediction(training_data, eval_data, kernel=kernel, gamma=gamma)

for kernel in ['poly', 'sigmoid']:
    for coef0 in  np.logspace(-2, 0.5, num=6):
        print_prediction(training_data, eval_data, kernel=kernel, coef0=coef0)

for kernel in kernels:
    for decision_function_shape in ['ovo', 'ovr']:
        print_prediction(training_data, eval_data, kernel=kernel, decision_function_shape=decision_function_shape)
        
for kernel in ['poly']:
    for degree in range(2, 5):
        print_prediction(training_data, eval_data, kernel=kernel, degree=degree)

# Findings
## Influnce of number of training data
The more training data is presented, the less likeli